In [90]:
from imutils import contours
import numpy as np
import argparse
import cv2
import myutils

In [91]:
img = cv2.imread('ocr_a_reference.png')

In [92]:
FIRST_NUMBER={
    '3':'American Express',
    '4':'Visa',
    '5':'MasterCard',
    '6':'Discover Card'
}

In [93]:
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows();

In [95]:
cv_show('img',img)

In [172]:
#canny
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#threshold
ret,thresh = cv2.threshold(gray,10,255,cv2.THRESH_BINARY_INV)
# cv_show('res',ret)
contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
draw_img = img.copy()
res = cv2.drawContours(draw_img,contours,-1,(0,0,255),1)
cv_show('res',draw_img)

In [173]:
#enumerate
digits = {};
refCnts = myutils.sort_contours(contours)[0]

for (i,c) in enumerate(refCnts):
    x,y,w,h = cv2.boundingRect(c);
    roi = thresh[y:y+h,x:x+w]
    roi = cv2.resize(roi,(57,88))
    cv_show('roi',roi)
    digits[i] = roi
    
    

In [ ]:
rectkernel = cv2.getStructuringElement(cv2.MORPH_RECT,(9,3))
sqkernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))

In [200]:
card_img = cv2.imread('images/credit_card_02.png')
card_img = myutils.resize(card_img,width=300)
card_img= cv2.GaussianBlur(card_img,(5,5),1);

gray_card = cv2.cvtColor(card_img,cv2.COLOR_BGR2GRAY)

tophat = cv2.morphologyEx(gray_card,cv2.MORPH_TOPHAT,rectkernel)
cv_show('tophat',tophat)

In [201]:
# sobel_res = cv2.Sobel(tophat,ddepth = cv2.CV_32F,dx = 1,dy = 0,ksize = -1)
# gradX = np.absolute(sobel_res)
# (minval,maxval) = (np.min(gradX),np.max(gradX))
# gradX = (255*((gradX-minval)/(maxval-minval)))
# gradX = gradX.astype("uint8")
gradX = cv2.Laplacian(tophat,cv2.CV_32F,ksize = 3)
gradX = cv2.convertScaleAbs(gradX)
cv_show('gradX',gradX)

In [202]:
# closing = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectkernel)
closing = cv2.dilate(gradX,rectkernel,iterations=1)
cv_show('closing',closing)
# dilation = cv2.dilate(closing,rectkernel,iterations = 1)
# cv_show("dilation",dilation)

In [203]:
#thresh
ret2,th2 = cv2.threshold(closing,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv_show('ret2',th2)

In [204]:
closing = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, sqkernel)

contours,hierarchy = cv2.findContours(closing.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
draw_img = card_img.copy()
res = cv2.drawContours(draw_img,contours,-1,(0,0,255),3)
cv_show('res',draw_img)

In [205]:
locs = []
for (i,cnt) in enumerate(contours):
    (x,y,w,h) = cv2.boundingRect(cnt)
    ar = w/float(h)
    print(x,y,w,h)
    if ar > 2.5 and ar < 4.0:
        if  (w>40 and w<86) and (h>10 and h<45):
            locs.append([x,y,w,h])
locs = sorted(locs,key = lambda x:x[0])
print("size is ",len(locs))


28 154 86 14
219 151 75 26
91 109 118 18
29 109 57 18
216 108 56 20
224 78 59 14
189 15 93 45
size is  3


In [198]:
tmp = card_img.copy()
for (i,(gx,gy,gw,gh)) in enumerate(locs):
    groupoutput = []
    #[c,r], c = column,r = row
    group = gray_card[gy-5:gy+gh+5,gx-5:gx+gw+5]
    
    group = cv2.threshold(group,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    contours,hierarchy = cv2.findContours(group.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    draw_img = card_img.copy()
    
    digits_cnt = myutils.sort_contours(contours)[0]
    
    for (i,c) in enumerate(digits_cnt):
        x,y,w,h = cv2.boundingRect(c);
        roi = group[y:y+h,x:x+w]
        roi = cv2.resize(roi,(57,88))
            
        scores = []
        
        for(digit,digitROI) in digits.items():
            result = cv2.matchTemplate(roi,digitROI,cv2.TM_CCOEFF);
            (_,maxval,_,_) = cv2.minMaxLoc(result);
            
            scores.append(maxval);
        print("max is ",str(np.argmax(scores)))
        groupoutput.append(str(np.argmax(scores)))
    
    
    cv2.rectangle(tmp, (gx - 5, gy - 5),
        (gx + gw + 5, gy + gh + 5), (0, 0, 255), 1)
    cv2.putText(tmp, "".join(groupoutput), (gx, gy - 15),
        cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
    
    output.extend(groupoutput)


print("Credit Card #: {}".format("".join(output)))        
cv2.imshow("Image", tmp)

cv2.waitKey(0)

max is  4
max is  0
max is  2
max is  0
max is  5
max is  6
max is  7
max is  8
max is  7
max is  7
max is  5
max is  1
Credit Card #: 11114441478781411111444147878141111144414787814140001234567890104020340002345678402034000234567840203400023456784020340002345678402034000234567840204020402056787751402056787751


113